In [27]:
import os, sys
import snntorch as snn 
import torch
from torch import nn

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from config import *
from src.dataset import *

# Convolutional SNN

Краткий туториал по устройству свертки в импульснах нейронных сетях.

## Импорт датасета

In [28]:
emg, label = folder_extract('../'+FOLDER_PATH, exercises=EXERCISES, myo_pref=MYO_PREF)
all_g = gestures(emg, label, targets=GESTURE_INDEXES_MAIN)

train_g, test_g = train_test_split(all_g, split_size=0.2, rand_seed=GLOBAL_SEED)

X_train_raw, y_train = apply_window(train_g, window=WINDOW_SIZE, step=STEP_SIZE)
X_test_raw,  y_test  = apply_window(test_g,  window=WINDOW_SIZE, step=STEP_SIZE)

## Стандартизация и подготовка к размерности модели

In [29]:
means = X_train_raw.mean(axis=(0, 2))       # (channels,)
stds  = X_train_raw.std(axis=(0, 2)) + 1e-8

def standardize(X):
    return (X - means[None,:,None]) / stds[None,:,None]

Xs_train = standardize(X_train_raw)
Xs_test = standardize(X_test_raw)

In [30]:
def prepare(X):
    Xt = np.transpose(X, (0, 2, 1))   # [N, window, channels]
    sel = Xt[..., CHANNELS]           # отбор каналов
    return sel[..., np.newaxis].astype(np.float32)

X_train = prepare(Xs_train)  # Готовые данные
X_test = prepare(Xs_test)

In [31]:
X_train = torch.tensor(X_train, dtype=torch.float32).permute(0, 3, 1, 2)

In [32]:
X_train.shape    # (кол-во окон, 1, размерность окна, количество каналов)

torch.Size([32941, 1, 52, 8])

## Определение модели

In [33]:
spike_grad = snn.surrogate.fast_sigmoid(slope=25)
beta = 0.5
num_steps = 50

In [34]:
model = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3),
                      nn.MaxPool2d(kernel_size=2),
                      snn.Leaky(beta=0.95, spike_grad=spike_grad, init_hidden=True)
                      )

In [35]:
model(X_train[0])

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]], grad_fn=<MulBackward0>)